In [8]:
import sqlite3
import pandas as pd

db_path = 'hot100.db'

table_name = 'hot100'

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql(f"SELECT * FROM {table_name};", conn)

df.head()

,id,artist,song,count,youtube_url,uploader,duration,views,like_count,release_date,thumbnail,tags,description
0,d0f9b947-d9ac-400b-b4dd-25534b30370f,Lady Gaga & Bruno Mars,Die With A Smile,0,https://www.youtube.com/watch?v=kPa7bsKwL-c,Lady Gaga,252,819428697,9289800,,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,"[""Lady Gaga"", ""Bruno Mars"", ""Interscope"", ""Pop""]",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...
1,0200f2af-7b0f-4f87-ae4e-fea76fe1305f,Kendrick Lamar,Not Like Us,0,https://www.youtube.com/watch?v=H58vbez_m4E,Kendrick Lamar,354,289684537,6405687,,https://i.ytimg.com/vi/H58vbez_m4E/maxresdefau...,"[""Kendrick Lamar"", ""Kendrick Lamar, under excl...",Kendrick Lamar “Not Like Us”\nDirected by Dave...
2,5d2a6c53-a61c-4024-a173-fd4d39dfe427,Kendrick Lamar Featuring Lefty Gunplay,TV Off,4,https://www.youtube.com/watch?v=4ILLpWbgESM,Certified Filmmaker,221,1092830,14234,20241221,https://i.ytimg.com/vi/4ILLpWbgESM/sddefault.j...,"[""Kendrick Lamar"", ""tv off"", ""GNX"", ""Music"", ""...",#kendricklamar #tvoff #gnx #music #musicvideo ...
3,34ce80c8-b2de-41e3-a721-bffb69e15c07,Shaboozey,A Bar Song (Tipsy),0,https://www.youtube.com/watch?v=t7bQwwqW-Hc,Shaboozey,173,204128472,1625863,,https://i.ytimg.com/vi/t7bQwwqW-Hc/maxresdefau...,"[""Shaboozey"", ""Bar"", ""Song"", ""(Tipsy)"", ""[Offi...","Listen to the single “A Bar Song (Tipsy)"". Out..."
4,e3201b4d-5a39-45b0-8bf3-0e5eeff9894a,ROSE & Bruno Mars,APT.,0,https://www.youtube.com/watch?v=ekr2nIex040,ROSÉ,173,1298483960,14699729,20241018,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,"[""YG Entertainment"", ""YG"", ""\uc640\uc774\uc9c0...",ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...


In [9]:
df.drop(['youtube_url', 'release_date','thumbnail','description'], axis=1, inplace=True)
df.head()

,id,artist,song,count,uploader,duration,views,like_count,tags
0,d0f9b947-d9ac-400b-b4dd-25534b30370f,Lady Gaga & Bruno Mars,Die With A Smile,0,Lady Gaga,252,819428697,9289800,"[""Lady Gaga"", ""Bruno Mars"", ""Interscope"", ""Pop""]"
1,0200f2af-7b0f-4f87-ae4e-fea76fe1305f,Kendrick Lamar,Not Like Us,0,Kendrick Lamar,354,289684537,6405687,"[""Kendrick Lamar"", ""Kendrick Lamar, under excl..."
2,5d2a6c53-a61c-4024-a173-fd4d39dfe427,Kendrick Lamar Featuring Lefty Gunplay,TV Off,4,Certified Filmmaker,221,1092830,14234,"[""Kendrick Lamar"", ""tv off"", ""GNX"", ""Music"", ""..."
3,34ce80c8-b2de-41e3-a721-bffb69e15c07,Shaboozey,A Bar Song (Tipsy),0,Shaboozey,173,204128472,1625863,"[""Shaboozey"", ""Bar"", ""Song"", ""(Tipsy)"", ""[Offi..."
4,e3201b4d-5a39-45b0-8bf3-0e5eeff9894a,ROSE & Bruno Mars,APT.,0,ROSÉ,173,1298483960,14699729,"[""YG Entertainment"", ""YG"", ""\uc640\uc774\uc9c0..."


In [10]:
import pandas as pd
import json
import re
import numpy as np

# Function to split artist names
def process_artists(artist_str):
    # Split by common separators
    if isinstance(artist_str, str):
        # Replace "Featuring" and other variants
        artist_str = re.sub(r'Featuring|Feat\.|Feat|ft\.', '&', artist_str)
        # Split by & or "with" 
        artists = re.split(r'\s*&\s*|\s+with\s+', artist_str)
        # Strip whitespace
        return [artist.strip() for artist in artists]
    return []

# Function to process tags
def process_tags(tags_str):
    if isinstance(tags_str, str):
        try:
            # Parse JSON string
            tags = json.loads(tags_str)
            # Extract and clean tags
            return [tag.lower().strip() for tag in tags]
        except json.JSONDecodeError:
            return []
    return []

# Apply transformations to DataFrame
df['artists_list'] = df['artist'].apply(process_artists)
df['tags_list'] = df['tags'].apply(process_tags)

# Create engagement metrics
df['engagement_ratio'] = df['like_count'] / df['views'].apply(lambda x: max(x, 1))  # Avoid division by zero

# Normalize popularity metrics (Min-Max scaling)
df['views_normalized'] = (df['views'] - df['views'].min()) / (df['views'].max() - df['views'].min())
df['likes_normalized'] = (df['like_count'] - df['like_count'].min()) / (df['like_count'].max() - df['like_count'].min())

# Calculate a popularity score
df['popularity_score'] = (df['views_normalized'] * 0.7) + (df['likes_normalized'] * 0.3)

In [4]:
df

,id,artist,song,count,uploader,duration,views,like_count,tags,artists_list,tags_list,engagement_ratio,views_normalized,likes_normalized,popularity_score
0,d0f9b947-d9ac-400b-b4dd-25534b30370f,Lady Gaga & Bruno Mars,Die With A Smile,0,Lady Gaga,252,819428697,9289800,"[""Lady Gaga"", ""Bruno Mars"", ""Interscope"", ""Pop""]","[Lady Gaga, Bruno Mars]","[lady gaga, bruno mars, interscope, pop]",0.011337,0.630755,0.631614,0.631013
1,0200f2af-7b0f-4f87-ae4e-fea76fe1305f,Kendrick Lamar,Not Like Us,0,Kendrick Lamar,354,289684537,6405687,"[""Kendrick Lamar"", ""Kendrick Lamar, under excl...",[Kendrick Lamar],"[kendrick lamar, kendrick lamar, under exclusi...",0.022113,0.222440,0.435222,0.286275
2,5d2a6c53-a61c-4024-a173-fd4d39dfe427,Kendrick Lamar Featuring Lefty Gunplay,TV Off,0,Certified Filmmaker,221,1092830,14234,"[""Kendrick Lamar"", ""tv off"", ""GNX"", ""Music"", ""...","[Kendrick Lamar, Lefty Gunplay]","[kendrick lamar, tv off, gnx, music, music vid...",0.013025,0.000000,0.000000,0.000000
3,34ce80c8-b2de-41e3-a721-bffb69e15c07,Shaboozey,A Bar Song (Tipsy),0,Shaboozey,173,204128472,1625863,"[""Shaboozey"", ""Bar"", ""Song"", ""(Tipsy)"", ""[Offi...",[Shaboozey],"[shaboozey, bar, song, (tipsy), [official, vis...",0.007965,0.156495,0.109743,0.142470
4,e3201b4d-5a39-45b0-8bf3-0e5eeff9894a,ROSE & Bruno Mars,APT.,0,ROSÉ,173,1298483960,14699729,"[""YG Entertainment"", ""YG"", ""\uc640\uc774\uc9c0...","[ROSE, Bruno Mars]","[yg entertainment, yg, 와이지, k-pop, blackpink, ...",0.011321,1.000000,1.000000,1.000000
5,5769a3ec-c3fb-4d8a-8158-530f0bb36c03,Billie Eilish,Birds Of A Feather,0,Billie Eilish,231,291883583,3098253,"[""Billie Eilish"", ""Darkroom/Interscope Records...",[Billie Eilish],"[billie eilish, darkroom/interscope records, a...",0.010615,0.224135,0.210004,0.219896
6,9e0a60b7-5a1e-482b-977e-11fe9530e305,Chappell Roan,Pink Pony Club,0,Chappell Roan,280,47093488,534182,"[""Chappell"", ""Roan"", ""Pink"", ""Pony"", ""Club"", ""...",[Chappell Roan],"[chappell, roan, pink, pony, club, ps/, island...",0.011343,0.035456,0.035406,0.035441
7,60121fff-67f6-4df4-9043-169ff5df788b,Teddy Swims,Lose Control,0,Teddy Swims,209,87469839,746312,"[""teddy swims"", ""lose control"", ""lose control ...",[Teddy Swims],"[teddy swims, lose control, lose control live ...",0.008532,0.066577,0.049850,0.061559
8,00e44842-deca-4649-a64d-2bff0019233b,Drake,Nokia,1,PARTYNEXTDOOR,241,4302399,62755,"[""Drake"", ""OVO Sound"", ""R&B""]",[Drake],"[drake, ovo sound, r&b]",0.014586,0.002474,0.003304,0.002723
9,eb0ecd02-fc71-4d9f-a043-a4da7a0e9815,Gracie Abrams,That's So True,1,stylizt,165,1690110,25953,[],[Gracie Abrams],[],0.015356,0.000460,0.000798,0.000562


In [13]:
# ------- Artist and Tag Affinity Calculation -------

def calculate_artist_affinity(df):
    """Calculate user affinity for artists based on play counts"""
    artist_play_counts = {}
    
    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        play_count = row['count']
        if play_count > 0:  # Only consider songs that have been played
            for artist in row['artists_list']:
                if artist in artist_play_counts:
                    artist_play_counts[artist] += play_count
                else:
                    artist_play_counts[artist] = play_count
    
    # Normalize artist play counts
    total_plays = sum(artist_play_counts.values()) or 1  # Avoid division by zero
    artist_affinity = {artist: count/total_plays for artist, count in artist_play_counts.items()}
    
    return artist_affinity

def calculate_tag_affinity(df):
    """Calculate user affinity for music tags/genres based on play counts"""
    tag_play_counts = {}
    
    # Iterate through rows
    for _, row in df.iterrows():
        play_count = row['count']
        if play_count > 0:
            for tag in row['tags_list']:
                if tag in tag_play_counts:
                    tag_play_counts[tag] += play_count
                else:
                    tag_play_counts[tag] = play_count
    
    # Normalize
    total_plays = sum(tag_play_counts.values()) or 1
    tag_affinity = {tag: count/total_plays for tag, count in tag_play_counts.items()}
    
    return tag_affinity

def score_by_artist_affinity(row, artist_affinity):
    """Score songs based on user's artist preferences"""
    score = 0
    for artist in row['artists_list']:
        score += artist_affinity.get(artist, 0)
    return score

def score_by_tag_affinity(row, tag_affinity):
    """Score songs based on user's genre/tag preferences"""
    score = 0
    for tag in row['tags_list']:
        score += tag_affinity.get(tag, 0)
    return score

# ------- Collaborative Filtering Component -------

def calculate_song_similarity(df):
    """Calculate similarity between songs based on artists and tags"""
    n_songs = len(df)
    similarity_matrix = np.zeros((n_songs, n_songs))
    
    for i in range(n_songs):
        for j in range(i, n_songs):
            if i == j:
                similarity_matrix[i][j] = 1.0
                continue
                
            # Get artists and tags for both songs
            artists_i = set(df.iloc[i]['artists_list'])
            artists_j = set(df.iloc[j]['artists_list'])
            
            tags_i = set(df.iloc[i]['tags_list'])
            tags_j = set(df.iloc[j]['tags_list'])
            
            # Calculate Jaccard similarity for artists and tags
            if len(artists_i) == 0 or len(artists_j) == 0:
                artist_sim = 0
            else:
                artist_sim = len(artists_i & artists_j) / len(artists_i | artists_j)
                
            if len(tags_i) == 0 or len(tags_j) == 0:
                tag_sim = 0
            else:
                tag_sim = len(tags_i & tags_j) / len(tags_i | tags_j)
            
            # Weight artist similarity more heavily than tag similarity
            final_sim = (artist_sim * 0.7) + (tag_sim * 0.3)
            
            # Make matrix symmetric
            similarity_matrix[i][j] = final_sim
            similarity_matrix[j][i] = final_sim
    
    return similarity_matrix

# ------- Main Recommendation System -------

def build_recommendation_model(df):
    """Build the complete recommendation model"""
    
    # 1. Create normalized count (handling zero division case)
    max_count = df['count'].max()
    min_count = df['count'].min()
    if max_count > min_count:
        df['count_normalized'] = (df['count'] - min_count) / (max_count - min_count)
    else:
        df['count_normalized'] = df['count'].apply(lambda x: 1.0 if x > 0 else 0.0)
    
    # 2. Calculate recency factor (more weight to recently played songs)
    df['recency_factor'] = df['count_normalized'] * 1.5  # Boost recent plays
    
    # 3. Calculate artist and tag affinity
    artist_affinity = calculate_artist_affinity(df)
    tag_affinity = calculate_tag_affinity(df)
    
    # 4. Apply the affinity scores
    df['artist_affinity_score'] = df.apply(lambda row: score_by_artist_affinity(row, artist_affinity), axis=1)
    df['tag_affinity_score'] = df.apply(lambda row: score_by_tag_affinity(row, tag_affinity), axis=1)
    
    # 5. Calculate song similarity matrix for collaborative filtering
    similarity_matrix = calculate_song_similarity(df)
    
    # 6. Collaborative filtering score - similar to what user has played
    def collaborative_score(idx):
        if df.loc[idx, 'count'] > 0:
            # If song is already played, collaborative score doesn't matter as much
            return 0.0
        
        # Find similarity with played songs
        collab_score = 0
        played_songs = df[df['count'] > 0].index
        
        if len(played_songs) == 0:
            return 0.0
        
        # Get similarities with played songs, weighted by play count
        for played_idx in played_songs:
            sim = similarity_matrix[idx][played_idx]
            play_count = df.loc[played_idx, 'count']
            collab_score += sim * play_count
        
        # Normalize by number of played songs
        return collab_score / len(played_songs)
    
    # Apply collaborative filtering
    df['collaborative_score'] = df.index.map(collaborative_score)
    
    # 7. Final blended recommendation score
    df['recommendation_score'] = (
        # Direct user interaction signals (50%)
        df['count_normalized'] * 0.4 +             # Play count is very important
        df['engagement_ratio'] * 0.1 +             # How engaging the content is
        
        # Personalization signals (35%)
        df['artist_affinity_score'] * 0.2 +        # Artist preference
        df['tag_affinity_score'] * 0.15 +          # Genre preference
        
        # Collaborative filtering (5%)
        df['collaborative_score'] * 0.05 +
        
        # Global popularity (10%)
        df['popularity_score'] * 0.1               # General popularity
    ) * (1 + df['recency_factor'] * 0.5)           # Recency boost
    
    # 8. Handle cold start (no user history)
    if df[df['count'] > 0].empty:
        # If no songs have been played, use popularity & engagement
        df['recommendation_score'] = (
            df['popularity_score'] * 0.7 +
            df['engagement_ratio'] * 0.3
        )
    
    # Store the dataframe for future predictions
    return df.sort_values('recommendation_score', ascending=False)

# ------- User Recommendation Functions -------

def get_recommendations(df, n=5, exclude_played=False):
    """Get top N song recommendations"""
    model_df = build_recommendation_model(df)
    
    if exclude_played:
        model_df = model_df[model_df['count'] == 0]
    
    # Get top N recommendations
    recommendations = model_df.head(n)
    
    # Format output as a user-friendly list
    result = []
    for i, (_, song) in enumerate(recommendations.iterrows()):
        result.append({
            'rank': i+1,
            'song': song['song'],
            'artist': song['artist'],
            'recommendation_score': song['recommendation_score'],
            'id': song['id']
        })
    
    return result

def get_recommendations_by_artist(df, artist, n=3):
    """Get top N song recommendations by a specific artist"""
    model_df = build_recommendation_model(df)
    
    # Filter by artist (case insensitive partial match)
    artist_df = model_df[model_df['artist'].str.lower().str.contains(artist.lower())]
    
    # Sort by recommendation score and return top N
    return artist_df.head(n)[['artist', 'song', 'recommendation_score']]

def get_recommendations_by_tag(df, tag, n=3):
    """Get top N song recommendations with a specific tag/genre"""
    model_df = build_recommendation_model(df)
    
    # Filter songs that have this tag
    tag_matches = model_df[model_df['tags_list'].apply(lambda tags: any(tag.lower() in t.lower() for t in tags))]
    
    # Sort by recommendation score and return top N
    return tag_matches.head(n)[['artist', 'song', 'recommendation_score']]

In [ ]:
# 1. Get general recommendations
top_recommendations = get_recommendations(df, n=10)
print("Top 10 Recommended Songs:")
for rec in top_recommendations:
    print(f"{rec['rank']}. {rec['artist']} - {rec['song']} (Score: {rec['recommendation_score']:.4f})")

# 2. Get recommendations excluding songs already played
new_recommendations = get_recommendations(df)
print("\nTop 5 New Songs to Try:")
for rec in new_recommendations:
    print(f"{rec['id']} {rec['rank']}. {rec['artist']} - {rec['song']}")

# 3. Get recommendations by artist
kendrick_recs = get_recommendations_by_artist(df, "Kendrick", n=3)
print("\nTop Kendrick Lamar Songs Recommended:")
print(kendrick_recs)

# 4. Get recommendations by genre/tag
pop_recs = get_recommendations_by_tag(df, "pop", n=3)
print("\nTop Pop Songs Recommended:")
print(pop_recs)

Top 10 Recommended Songs:
1. Kendrick Lamar Featuring Lefty Gunplay - TV Off (Score: 1.2465)
2. Kendrick Lamar - Squabble Up (Score: 0.9133)
3. Kendrick Lamar - Not Like Us (Score: 0.2283)
4. Drake - Nokia (Score: 0.1502)
5. Gracie Abrams - That's So True (Score: 0.1376)
6. ROSE & Bruno Mars - APT. (Score: 0.1011)
7. Lady Gaga & Bruno Mars - Die With A Smile (Score: 0.0642)
8. Benson Boone - Beautiful Things (Score: 0.0403)
9. Sabrina Carpenter - Espresso (Score: 0.0261)
10. Billie Eilish - Birds Of A Feather (Score: 0.0231)

Top 5 New Songs to Try:
0200f2af-7b0f-4f87-ae4e-fea76fe1305f 1. Kendrick Lamar - Not Like Us
e3201b4d-5a39-45b0-8bf3-0e5eeff9894a 2. ROSE & Bruno Mars - APT.
d0f9b947-d9ac-400b-b4dd-25534b30370f 3. Lady Gaga & Bruno Mars - Die With A Smile
d1f54212-1f19-450c-8479-a299b05ef991 4. Benson Boone - Beautiful Things
95504049-0bd2-4adc-9267-7faf080cde6e 5. Sabrina Carpenter - Espresso

Top Kendrick Lamar Songs Recommended:
                                    artist      